<div style="color:purple;background-color:green">
    hello

In [1]:
from operator import index
import pandas as pd
import os
import math
import seaborn as sns
import matplotlib.pyplot as plt

In this project there are four main goals:
1. to count how many lines are in a drawing
2. to calculate the length of each line and the mean pressure when computing each line
3. to check how many hand lifts were in each drawing
4. to check if the shape is open or closed (bonus)

#### 1. counting how many lines are in a drawing

In [2]:
def dist(x1, y1, x2, y2):
    return math.sqrt(pow(x1-x2,2) + pow(y1-y2,2))

In [3]:
def drop_close_values(lst):
    indexes_to_drop = []
    for i in range(len(lst)-1):
        if (lst[i+1]<lst[i]+9):
            indexes_to_drop.append(lst[i+1])
    new_list = [index for index in lst if index not in indexes_to_drop]
    return new_list

In [137]:
def len_from_point_to_point(df, a, b):
    return df[(df.index > a) & (df.index <= b)]['dist'].sum()

In [147]:
def find_lines_by_angle_and_dist(df, to_print):
    indexes = []
    lst = []
    df1 = pd.DataFrame({'Y_DIF': df[' Y'].diff()})
    df2 = pd.DataFrame({'X_DIF': df['X'].diff()})
    
    df_change = pd.concat([df1, df2, df[' Time'], df[' Pressure']], axis=1)
    df_change.drop(df_change.head(1).index, inplace=True)
    df_change = df_change[df_change['X_DIF'] != 0]
    df_angles = pd.DataFrame({'ANG': (df_change['Y_DIF']/df_change['X_DIF']).apply(lambda x: math.degrees(math.atan(x))), 
                             ' Time': df_change[' Time'], ' Pressure': df_change[' Pressure']})
    df_angles['diff'] = df_angles['ANG'].diff()
    df_angles.drop(df_angles.head(1).index, inplace=True)
    df_angles['idx'] = df_angles.index
    df_angles['ave_ang'] = df_angles['idx'].apply(lambda x: df_angles['ANG'][(df_angles.index > x) & (df_angles.index <= x+5)].mean())
    df_angles.drop(df_angles.tail(1).index, inplace=True)
    df_angles.drop(['idx'], axis=1, inplace=True)
    df_angles = df_angles.reset_index()
    
    for point in df_angles.index:
        if ((df_angles.loc[point, 'diff'] > 40) or (df_angles.loc[point, 'diff'] < -40)):
            if (point -1 < 0):
                continue
            if (abs(df_angles.loc[point-1, 'ANG'] - df_angles.loc[point, 'ave_ang']) > 40):
                indexes.append(df_angles.loc[point, 'index'])
    if (df['dist'].std()>2*df['dist'].median()):
        lst = df[df['dist'] > (df['dist'].median()+(df['dist'].std()/2))].index.values.tolist()
    
    indexes = sorted(indexes + lst)
    indexes = drop_close_values(indexes)
    
    
    indexes_to_drop = []
    flag = True
    
    while(flag == True):
        if (len(indexes)==0):
            break
        indexes_to_drop = []
        flag = False
        if (len_from_point_to_point(df, 0, indexes[0]) <= 16.0):
            indexes_to_drop.append(indexes[0])
            flag = True
        if (len_from_point_to_point(df, indexes[len(indexes)-1], len(df)-1) <= 16.0):
            indexes_to_drop.append(indexes[len(indexes)-1])
            flag = True
        for i in range(1, len(indexes)):
            #print(len(indexes))
            #print(indexes)
            p = len_from_point_to_point(df, indexes[i-1], indexes[i])
            #print(p, indexes[i-1], indexes[i], i+1)
            if (p <= 16.0):
                #print(i, i-1, dis, indexes[i], indexes[i-1], df.loc[indexes[i-1], 'X'], df.loc[indexes[i-1], ' Y'], df.loc[indexes[i], 'X'], df.loc[indexes[i], ' Y'])
                indexes_to_drop.append(indexes[i])
                flag = True
        #print(indexes)
        indexes = [index for index in indexes if index not in indexes_to_drop]
        
    
    #indexes = [index for index in indexes if index not in indexes_to_drop]
    
    num_of_lines = len(indexes) + 1
    
    if (to_print==False):
        return indexes
    
    print(indexes)
    #print(df['dist'].mean(), df['dist'].median(), df['dist'].std())
    #print(df_angles)
    print("Number of lines in shape is: ", num_of_lines)

In [148]:
def get_indexes_of_lines(df):
    return find_lines_by_angle_and_dist(df, False)

In [152]:
for dir in os.listdir("C:/אריאל/שנה א/סמסטר ב/הדמיית נתונים/Data_visualization_project/Working with pandas - kids drawings project/data"):
    if os.path.isdir("C:/אריאל/שנה א/סמסטר ב/הדמיית נתונים/Data_visualization_project/Working with pandas - kids drawings project/data/"+dir):
        full_dir = "C:/אריאל/שנה א/סמסטר ב/הדמיית נתונים/Data_visualization_project/Working with pandas - kids drawings project/data/" + dir + "/SimpleTest/"
        for file in os.listdir(full_dir):
            if file.endswith('.csv'):
                if (file == "1.csv"):
                    print(full_dir+file)
                    df = pd.read_csv(full_dir+file)
                    df.drop(df.tail(2).index, inplace=True) #last two lines appers only on the 'X' column and contains summary of drawing time
                    if (df.shape[0] < 2): #less than 2 rows means that the file is empty
                        continue
                    df['X'] = df['X'].astype('float64')
                    df[' Time'] = pd.to_datetime(df[' Time'])
                    df[' Pressure'] = df[' Pressure'] * 100 #in order to see things more clearly in the graphs
                    df['dist'] = (df['X'].diff()**2 + df[' Y'].diff()**2).apply(lambda x: math.sqrt(x)) #the distance between point to the previous one
                    
                    find_lines_length_by_index(df)
                    find_lines_pressure_by_index(df)
                    
                    #df.drop(df.head(1).index, inplace=True) #the first measurment dosen't have distance because it dosen't have a previous point
                    find_lines_by_angle_and_dist(df, True)
                    #print(get_indexes_of_lines(df))

C:/אריאל/שנה א/סמסטר ב/הדמיית נתונים/Data_visualization_project/Working with pandas - kids drawings project/data/7554/SimpleTest/1.csv
Length of line 1:  58.78354415645511
Length of line  2 :  42.1778682271858
Length of line  3 :  247.41867225691897
Average pressure of line 1:  26.01517771565636
Average pressure of line  2 :  36.86348449799321
Average pressure of line  3 :  36.16635557906381
[42, 73]
Number of lines in shape is:  3
C:/אריאל/שנה א/סמסטר ב/הדמיית נתונים/Data_visualization_project/Working with pandas - kids drawings project/data/7556/SimpleTest/1.csv
Total length:  1316.3869744997571
Average pressure:  59.30249269245661
[]
Number of lines in shape is:  1
C:/אריאל/שנה א/סמסטר ב/הדמיית נתונים/Data_visualization_project/Working with pandas - kids drawings project/data/7557/SimpleTest/1.csv
Total length:  155.25330775449657
Average pressure:  82.27659076093191
[]
Number of lines in shape is:  1
C:/אריאל/שנה א/סמסטר ב/הדמיית נתונים/Data_visualization_project/Working with panda

#### 2. calculating the length of each line and the mean pressure when computing each line:

In [128]:
def find_lines_length_by_index(df):
    #df is the dataframe, indexes is a list of the start/ end points of each line in the drawing (end of one is the start of another)
    indexes = get_indexes_of_lines(df)
    if (len(indexes)==0):
        print("Total length: ", df['dist'].sum())
        return
        
    elif (len(indexes)==1):
        print("Length of line 1: ", df[df.index <= indexes[0]]['dist'].sum())
        print("Length of line 2: ", df[df.index > indexes[0]]['dist'].sum())
        return
        
    for i in range(len(indexes)):
        if (i==0):
            print("Length of line 1: ", df[df.index <= indexes[0]]['dist'].sum())
        elif (i==len(indexes)-1):
            print("Length of line ", (i+1), ": ", df[(df.index > indexes[i-1]) & (df.index <= indexes[i])]['dist'].sum())
            print("Length of line ", len(indexes) + 1,": ", df[df.index > indexes[len(indexes)-1]]['dist'].sum())
        else:
            print("Length of line ", (i+1), ": ", df[(df.index > indexes[i-1]) & (df.index <= indexes[i])]['dist'].sum())

In [151]:
def find_lines_pressure_by_index(df):
    indexes = get_indexes_of_lines(df)
    if (len(indexes)==0):
        print("Average pressure: ", df[' Pressure'].mean())
        return
    
    elif (len(indexes)==1):
        print("Average pressure of line 1: ", df[df.index <= indexes[0]][' Pressure'].mean())
        print("Average pressure of line 2: ", df[df.index > indexes[0]][' Pressure'].mean())
        return
        
    for i in range(len(indexes)):
        if (i==0):
            print("Average pressure of line 1: ", df[df.index <= indexes[0]][' Pressure'].mean())
        elif (i==len(indexes)-1):
            print("Average pressure of line ", (i+1), ": ", df[(df.index > indexes[i-1]) & (df.index <= indexes[i])][' Pressure'].mean())
            print("Average pressure of line ", len(indexes) + 1,": ", df[df.index > indexes[len(indexes)-1]][' Pressure'].mean())
        else:
            print("Average pressure of line ", (i+1), ": ", df[(df.index > indexes[i-1]) & (df.index <= indexes[i])][' Pressure'].mean())

#### 3. checking how many hand lifts were in each drawing

In [ ]:
def find_hand_off_by_pressure(df):
    ave_pressure = df[' Pressure'].mean()
    min_pressure = df[' Pressure'].min()
    max_pressure = df[' Pressure'].max()
    smallest_values = df[df[' Pressure'] < (min_pressure + max_pressure)/2].sort_index()
    smallest_values['idx'] = smallest_values.index
    smallest_values['idx_shift'] = smallest_values.shift()['idx']
    smallest_values.drop(smallest_values.head(1).index,inplace=True)
    num_of_hand_lifts = (smallest_values['idx']-smallest_values['idx_shift']).apply(lambda x: 0 if x==1.0 else 1).sum() + 1
    print("The number of hand lifts computed by pressure is: ", num_of_hand_lifts)

#### 4. checking if the shape is open or closed

In [ ]:
def find_line(df, indexes, point):
    my_line = 0
    sta_idx = 0
    end_idx = 0
    if (point < indexes[0].index):
        my_line = 0
        sta_idx = 0
        end_idx = indexes[0].index
    elif (point > indexes[len(indexes)-1]):
        my_line = len(indexes) + 1
        sta_idx = indexes[len(indexes)-1].index
        end_idx = len(df)
    else:
        for line in range(1, len(indexes)):
            if (point > indexes[line-1].index and point < indexes[line].index):
                my_line = line + 1
                sta_idx = indexes[line-1].index
                end_idx = indexes[line].index
    return my_line, sta_idx, end_idx   

In [ ]:
def which_lines_are_meetings(df):
    indexes = get_indexes_of_lines(df)
    ave_radius = 0
    meet_list = []
    my_line = 0
    skipin = 0
    skipout = 0
    if (len(indexes)==0):
        ave_radius = df['dist'].mean()
        for point in df.index:
            if (skipout > 0):
                skipout = skipout - 1
                continue
            for point2 in df.index:
                if (skipin > 0):
                    skipin = skipin - 1
                    continue
                elif ((point2>=point-5) and (point2<=point+5)):
                    continue
                elif (dist(df[point]['X'], df[point][' Y'], df[point2]['X'], df[point2][' Y']) < ave_radius):
                    meet_list.append([1, 1, point, point2])
                    skipin = 3
                    skipout = 3
                else:
                    continue
    else:
        for point in df.index:
            if (skipout > 0):
                skipout = skipout - 1
                continue
            my_line, start_line, end_line = find_line(df, indexes, point)
            ave_radius = df[(df.index >= start_line) & (df.index < end_line)]['dist'].mean()
            for point2 in df.index:
                if (skipin > 0):
                    skipin = skipin - 1
                    continue
                elif ((point2>=point-5) and (point2<=point+5)):
                    continue
                elif (dist(df[point]['X'], df[point][' Y'], df[point2]['X'], df[point2][' Y']) < ave_radius):
                    point2_line, start2, end2 = find_line(df, indexes, point2)
                    meet_list.append([my_line, point2_line, point, point2])
                    skipin = 3
                    skipout = 3
                else:
                    continue

In [ ]:
def is_shape_open(df, indexes):
    #df is the dataframe, indexes is the list of start/end points of each line
    #does the average delta x needs to be to each line or to the whole shape?
    #averageradius = df['dist'].mean()
    #what if the shape is open and contains multiple lines that do not connect (such as picture number 9)
    boolian = False
    if (len(indexes)==0):
        
    for point in df.rows:
        for point2 in df.rows:
            if (point==point2):
                continue
            elif (dist(df[point]['X'], df[point][' Y'], df[point2]['X'], df[point2][' Y']) < averageradius):
                boolian = True
                break
    return boolian

testing functions:

In [ ]:
for dir in os.listdir("C:/אריאל/שנה א/סמסטר ב/הדמיית נתונים/Data_visualization_project/Working with pandas - kids drawings project/data"):
    if os.path.isdir("C:/אריאל/שנה א/סמסטר ב/הדמיית נתונים/Data_visualization_project/Working with pandas - kids drawings project/data/"+dir):
        full_dir = "C:/אריאל/שנה א/סמסטר ב/הדמיית נתונים/Data_visualization_project/Working with pandas - kids drawings project/data/" + dir + "/SimpleTest/"
        for file in os.listdir(full_dir):
            if file.endswith('.csv'):
                if (file == "12.csv"):
                    print(full_dir+file)
                    df = pd.read_csv(full_dir+file)
                    df.drop(df.tail(2).index, inplace=True) #last two lines appers only on the 'X' column and contains summary of drawing time
                    if (df.shape[0] < 2): #less than 2 rows means that the file is empty
                        continue
                    df['X'] = df['X'].astype('float64')
                    df[' Time'] = pd.to_datetime(df[' Time'])
                    df[' Pressure'] = df[' Pressure'] * 100 #in order to see things more clearly in the graphs
                    df['dist'] = (df['X'].diff()**2 + df[' Y'].diff()**2).apply(lambda x: math.sqrt(x)) #the distance between point to the previous one
                    #df.drop(df.head(1).index, inplace=True) #the first measurment dosen't have distance because it dosen't have a previous point
                    
                    #find_hand_off_by_pressure(df)
                    #find_lines_length_by_index(df)
                    
                    
                    sns.lineplot(data=df,y='dist', x=' Time')
                    sns.lineplot(data=df,y=' Pressure', x=' Time')
                    plt.show() #distance-Time graph with added line for pressure